In [32]:
import cv2
import os
import glob
import shutil
from tqdm.notebook import tqdm
from PIL import Image
import numpy as np

In [33]:
import tensorflow as tf

In [34]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard,ModelCheckpoint,EarlyStopping
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten ,Dropout,LeakyReLU

In [35]:
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
%matplotlib inline
from tensorflow.keras.applications import VGG16

In [6]:
rootdir = 'D:/data-anal/marytalk/crop/'

In [7]:
#헤어 카테고리 리스트
categories = os.listdir(rootdir) #14 categories
nb_classes = len(categories) #14

In [8]:
#카테고리별 이미지 수 확인
for f in categories:
    path = rootdir+f
    print(f+' :',len(next(os.walk(path))[2]))

baby_perm : 1415
bob_cut : 663
buzz_cut : 326
c_curl_perm : 543
dandy_cut : 464
dreadlocks : 1042
hippie_perm : 482
hush_cut : 478
layered_cut : 317
mohican : 440
part_perm : 923
pomade : 1324
wave_perm : 208
wolf_cut : 798


In [ ]:
def imageResize(rootdir: str, folder_list: list, height:int, width:int):
    for c in folder_list :
        path = rootdir+c
        files = next(os.walk(path))[2]
        
        for f in files :
            resized_path = f'resized/{c}/'
            if not os.path.isdir(resized_path):
                os.makedirs(resized_path)
            
            img = cv2.imread(path+'/'+f)
            shape_ = img.shape
            if shape_[0]<height or shape_[1]<width :
                resized = cv2.resize(img, dsize=(height, width)
                                     , interpolation=cv2.INTER_CUBIC)
            else :
                resized = cv2.resize(img, dsize=(height, width)
                                     , interpolation=cv2.INTER_AREA)
            
            cv2.imwrite(resized_path+f,resized)

In [ ]:
imageResize(rootdir, categories, 224, 224)

In [9]:
data_generator = ImageDataGenerator(
        rotation_range=10 #이미지 회전 범위
        ,width_shift_range=0.15 #가로 이동 범위
        ,height_shift_range=0.15 #세로 이동 범위
        ,shear_range=0.05 #이미지 변형
        ,zoom_range=0.1 #확대 범위
        ,horizontal_flip=True
        ,vertical_flip=False
        ,rescale=1./255
        ,fill_mode='wrap' #공간 채우는 방식
        ,validation_split=0.2
        )

In [10]:
train_generator = data_generator.flow_from_directory(
        'resized'
        , subset='training'
        ,target_size=(224, 224)
        , batch_size=50
        ,classes=categories
        ,class_mode='categorical'
        )

val_generator = data_generator.flow_from_directory(
        'resized'
        , subset='validation'
        ,target_size=(224, 224)
        , batch_size=50
        ,classes=categories
        ,class_mode='categorical'
        )

Found 7545 images belonging to 14 classes.
Found 1878 images belonging to 14 classes.


In [11]:
base_model = VGG16(
    weights = 'imagenet'
    ,include_top =False
    , input_shape=(224,224,3)
)

In [14]:
x = base_model.output
x = Flatten()(x)
predictions=Dense(nb_classes,activation='softmax')(x)

In [15]:
model = Model(inputs=base_model.input, outputs=predictions)

In [16]:
for layer in base_model.layers:
    layer.trainable =False

In [17]:
# 훈련할 레이어 확인
for layer in base_model.layers:
    print(layer,layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x0000011F908AA588> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000011F8F567688> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000011F908BCEC8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000011F9089CB08> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000011F908D1F88> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000011F90AA7808> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000011F90AB0888> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000011F90AA6708> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000011F90E61F08> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000011F908AF7C8> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000011F90A86208> False
<tensorflow.python.

In [18]:
model.compile(loss='categorical_crossentropy'
             ,optimizer='adam'
             ,metrics=['accuracy']
             )
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [28]:
train_generator.class_indices

{'baby_perm': 0,
 'bob_cut': 1,
 'buzz_cut': 2,
 'c_curl_perm': 3,
 'dandy_cut': 4,
 'dreadlocks': 5,
 'hippie_perm': 6,
 'hush_cut': 7,
 'layered_cut': 8,
 'mohican': 9,
 'part_perm': 10,
 'pomade': 11,
 'wave_perm': 12,
 'wolf_cut': 13}

In [31]:
# 모델 저장
MODEL_DIR ='./model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

# modelpath='./model/{epoch:02d}-{val_loss:.4f}.hdf5'
modelpath='./model/{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath=modelpath,monitor='val_loss',verbose=1,save_best_only=True)
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10)

# with tf.device('/device:gpu:0'):
# 모델의 실행
model.fit_generator(
        train_generator,
        steps_per_epoch=1000,
        epochs=50,
        validation_data=val_generator,
        verbose=1,
        class_weight = "balanced",
        callbacks=[early_stopping_callback,checkpointer]
        )

# 테스트 정확도 출력
scores = model.evaluate_generator(val_generator, steps=5)
print("\n Test Accuracy: %.4f"%(model.metrics_names[1], scores[1]*100))

Epoch 1/50
 999/1000 [============================>.] - ETA: 24s - loss: 0.8220 - accuracy: 0.7291
Epoch 00001: val_loss improved from inf to 1.01774, saving model to ./model/01-1.0177.hdf5
1000/1000 [==============================] - 24869s 25s/step - loss: 0.8220 - accuracy: 0.7291 - val_loss: 1.0177 - val_accuracy: 0.7077
Epoch 2/50
 999/1000 [============================>.] - ETA: 24s - loss: 0.5977 - accuracy: 0.8025
Epoch 00002: val_loss improved from 1.01774 to 1.01687, saving model to ./model/02-1.0169.hdf5
1000/1000 [==============================] - 24724s 25s/step - loss: 0.5977 - accuracy: 0.8026 - val_loss: 1.0169 - val_accuracy: 0.7220
Epoch 3/50
 204/1000 [=====>........................] - ETA: 5:32:25 - loss: 0.5507 - accuracy: 0.8167

KeyboardInterrupt: 

In [29]:
tf.config.set_soft_device_placement(True)

In [30]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2074894888293703322
]
